# text2rdf

## Imports

In [1]:
# General purpose
import os
import glob
import random
import xml.etree.ElementTree as ET

# PyTorch 
import torch
import torch.nn as nn

# Bert
from transformers import BertTokenizer, BertModel, BertConfig

## Preprocessing

### Select Data-sub-set (given restricted nr of triples per sentence)

In [2]:
# How many triples to train and test system on (min: 1, max: 7)
MIN_NUM_TRIPLES = 1
MAX_NUM_TRIPLES = 2

In [3]:
# Set paths where to retrieve data from
DS_BASE_PATH = './WebNLG/'

TRAIN_PATH = DS_BASE_PATH + 'train/'
TEST_PATH = DS_BASE_PATH + 'dev/'

TRAIN_DIRS = [ TRAIN_PATH + str(i) + 'triples/' for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1) ]
TEST_DIRS  = [ TEST_PATH  + str(i) + 'triples/' for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1) ]

In [4]:
# Print selected directories
print('Train dirs:', TRAIN_DIRS)
print('Test  dirs:', TEST_DIRS)

Train dirs: ['./WebNLG/train/1triples/', './WebNLG/train/2triples/']
Test  dirs: ['./WebNLG/dev/1triples/', './WebNLG/dev/2triples/']


### Load Data

#### Load Settings (do not touch)

In [5]:
originaltripleset_index = 0  # Index of 'originaltripleset' attribute in XML entry
modifiedtripleset_index = 1  # Index of 'modifiedtripleset' attribute in XML entry
first_lexical_index = 2      # Index as of which verbalizations of RDF triples start in entry

#### Train Data

In [6]:
# Usage of train: train[target_nr_triples][element_id]['target_attribute']
train = [[] for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1)]

# Documents how many entries there are per number of triples
train_stats = [0 for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1)]

# Iterate through all files per number of triples and per category and load data
for i, d in enumerate(TRAIN_DIRS):
    nr_triples = list(range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1))[i]
    
    for filename in glob.iglob(d+'/**', recursive=False):
        if os.path.isfile(filename): # Filter dirs
            #print('File:', filename)
            
            tree = ET.parse(filename)
            root = tree.getroot()
            
            entries = root[0]
            train_stats[nr_triples-MIN_NUM_TRIPLES] += len(entries)
            
            for entry in entries:
                #print('Original triple set: ', entry[originaltripleset_index])
                #print('Modified triple set: ', entry[modifiedtripleset_index])
                
                modified_triple_set = entry[modifiedtripleset_index]
                
                for triple in modified_triple_set:
                    
                    verbalizations = entry[first_lexical_index:]
                
                    for verbalization in verbalizations:
                        #print('Text:', verbalization, verbalization.tag, verbalization.attrib, verbalization.text)
                        #print('Trip:', triple, triple.tag, triple.attrib, triple.text)
                        
                        train[i].append({ 'category': entry.attrib['category'],
                                          'id': entry.attrib['eid'],
                                          'triple_cnt': nr_triples,
                                          'text': verbalization.text,
                                          'triple': [text.strip() for text in triple.text.split('|')]
                                        })
                        
print(train)
print(train_stats)

[[{'category': 'Food', 'id': 'Id1', 'triple_cnt': 1, 'text': 'Ajoblanco originates from the country of Spain.', 'triple': ['Ajoblanco', 'country', 'Spain']}, {'category': 'Food', 'id': 'Id1', 'triple_cnt': 1, 'text': 'Ajoblanco is from Spain.', 'triple': ['Ajoblanco', 'country', 'Spain']}, {'category': 'Food', 'id': 'Id2', 'triple_cnt': 1, 'text': 'Ajoblanco has almond as one of its ingredients.', 'triple': ['Ajoblanco', 'ingredient', 'Almond']}, {'category': 'Food', 'id': 'Id2', 'triple_cnt': 1, 'text': 'Almond is an ingredient in ajoblanco.', 'triple': ['Ajoblanco', 'ingredient', 'Almond']}, {'category': 'Food', 'id': 'Id3', 'triple_cnt': 1, 'text': 'Bread is an ingredient of Ajoblanco.', 'triple': ['Ajoblanco', 'ingredient', 'Bread']}, {'category': 'Food', 'id': 'Id4', 'triple_cnt': 1, 'text': 'An ingredient of Ajoblanco is garlic.', 'triple': ['Ajoblanco', 'ingredient', 'Garlic']}, {'category': 'Food', 'id': 'Id4', 'triple_cnt': 1, 'text': 'Garlic is an ingredient used in Ajoblanco

#### Test Data

In [7]:
# Usage of test: test[target_nr_triples][element_id]['target_attribute']
test = [[] for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1)]

# Documents how many entries there are per number of triples
test_stats = [0 for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1)]

# Iterate through all files per number of triples and per category and load data
for i, d in enumerate(TEST_DIRS):
    nr_triples = list(range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1))[i]
    
    for filename in glob.iglob(d+'/**', recursive=False):
        if os.path.isfile(filename): # Filter dirs
            #print('File:', filename)
            
            tree = ET.parse(filename)
            root = tree.getroot()
            
            entries = root[0]
            test_stats[nr_triples-MIN_NUM_TRIPLES] += len(entries)
            
            for entry in entries:
                #print('Original triple set: ', entry[originaltripleset_index])
                #print('Modified triple set: ', entry[modifiedtripleset_index])
                
                modified_triple_set = entry[modifiedtripleset_index]
                
                for triple in modified_triple_set:
                    
                    verbalizations = entry[first_lexical_index:]
                
                    for verbalization in verbalizations:
                        #print('Text:', verbalization, verbalization.tag, verbalization.attrib, verbalization.text)
                        #print('Trip:', triple, triple.tag, triple.attrib, triple.text)
                        
                        test[i].append({ 'category': entry.attrib['category'],
                                          'id': entry.attrib['eid'],
                                          'triple_cnt': nr_triples,
                                          'text': verbalization.text,
                                          'triple': [text.strip() for text in triple.text.split('|')]
                                        })
                        
print(test)
print(test_stats)

[[{'category': 'ComicsCharacter', 'id': 'Id1', 'triple_cnt': 1, 'text': '\n        ', 'triple': ['Asterix_(character)', 'creators', 'René_Goscinny']}, {'category': 'ComicsCharacter', 'id': 'Id1', 'triple_cnt': 1, 'text': 'Asterix was created by Rene Goscinny.', 'triple': ['Asterix_(character)', 'creators', 'René_Goscinny']}, {'category': 'ComicsCharacter', 'id': 'Id1', 'triple_cnt': 1, 'text': 'The creator of Asterix (comics character) is René Goscinny.', 'triple': ['Asterix_(character)', 'creators', 'René_Goscinny']}, {'category': 'ComicsCharacter', 'id': 'Id1', 'triple_cnt': 1, 'text': 'The comic character Asterix, was created by René Goscinny.', 'triple': ['Asterix_(character)', 'creators', 'René_Goscinny']}, {'category': 'ComicsCharacter', 'id': 'Id2', 'triple_cnt': 1, 'text': '\n        ', 'triple': ['Auron_(comics)', 'creator', 'Marv_Wolfman']}, {'category': 'ComicsCharacter', 'id': 'Id2', 'triple_cnt': 1, 'text': 'The comic character Auron was created by Marv Wolfman.', 'triple'

#### Spilt Train Data into Train and Dev (for intermindiate validation throughout training)

In [8]:
# Percentage of train data reserved for validation throughout training
dev_percentage = 0.15

In [9]:
# Init dev dataset
dev = [[] for i in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1)]

# Sample number of dev instances per number of triples
dev_stats = [int(dev_percentage * train_stats[i]) for i in range(0, MAX_NUM_TRIPLES+1-MIN_NUM_TRIPLES)]

print('Samples per nr of triples:', dev_stats)

# Sample indices to be reserved for dev dataset for each nr of triples
dev_indices = [random.sample(range(0, len(train[i])), dev_stats[i]) for i in range(0, MAX_NUM_TRIPLES+1-MIN_NUM_TRIPLES)]
for i in range(len(dev_indices)):
    dev_indices[i].sort(reverse=True)

# Copy selected dev-entries into dev & delete all duplicates/related entries from train dataset
for nr_triples in range(0, MAX_NUM_TRIPLES+1-MIN_NUM_TRIPLES):
    
    # Iterate through all indices reserved for validation set (per nr of triples)
    for index in dev_indices[nr_triples]:
        
        # Select index'th train entry (to become dev/validation data)
        selected_entry = train[nr_triples][index]
        
        # Extract indentifying attributes
        entrys_category = selected_entry['category']
        entrys_idx = selected_entry['id']
        
        # Put selected entry into dev set
        dev[nr_triples].append(selected_entry)
        
        # Find all entries of matching index & category and remove them from train data
        for entry in train[nr_triples]:
            if entry['id'] == entrys_idx and entry['category'] == entrys_category:
                train[nr_triples].remove(entry)
                
print(dev)

Samples per nr of triples: [268, 181]
[[{'category': 'University', 'id': 'Id49', 'triple_cnt': 1, 'text': 'The School of Business and Social Sciences at the Aarhus University is affiliated to European University Association.', 'triple': ['Aarhus_University,_School_of_Business_and_Social_Sciences', 'affiliations', 'European_University_Association']}, {'category': 'University', 'id': 'Id46', 'triple_cnt': 1, 'text': 'One of the ethnic groups in Romania are the Germans of Romania.', 'triple': ['Romania', 'ethnicGroups', 'Germans_of_Romania']}, {'category': 'University', 'id': 'Id41', 'triple_cnt': 1, 'text': 'The Ganges is a river in India.', 'triple': ['India', 'river', 'Ganges']}, {'category': 'University', 'id': 'Id31', 'triple_cnt': 1, 'text': 'The Acharya Institute of Technology offers sports including tennis.', 'triple': ['Acharya_Institute_of_Technology', 'sportsOffered', 'Tennis']}, {'category': 'University', 'id': 'Id31', 'triple_cnt': 1, 'text': 'Acharya Institute of Technology 

#### Print Stats

In [10]:
print('Minimal number of triples:', MIN_NUM_TRIPLES)
print('Maximal number of triples:', MAX_NUM_TRIPLES)

print()

print('Training: ')
for nr_triples in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1):
    print('Given %i triples per sentence: ' % nr_triples)
    print('Number of combinations of triples and verbalizations:', len(train[nr_triples-MIN_NUM_TRIPLES]))

print()

print('Dev: ')
for nr_triples in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1):
    print('Given %i triples per sentence: ' % nr_triples)
    print('Number of combinations of triples and verbalizations:', len(dev[nr_triples-MIN_NUM_TRIPLES]))

print()

print('Testing: ')
for nr_triples in range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1):
    print('Given %i triples per sentence: ' % nr_triples)
    print('Number of combinations of triples and verbalizations:', len(test[nr_triples-MIN_NUM_TRIPLES]))

Minimal number of triples: 1
Maximal number of triples: 2

Training: 
Given 1 triples per sentence: 
Number of combinations of triples and verbalizations: 4204
Given 2 triples per sentence: 
Number of combinations of triples and verbalizations: 7037

Dev: 
Given 1 triples per sentence: 
Number of combinations of triples and verbalizations: 268
Given 2 triples per sentence: 
Number of combinations of triples and verbalizations: 181

Testing: 
Given 1 triples per sentence: 
Number of combinations of triples and verbalizations: 575
Given 2 triples per sentence: 
Number of combinations of triples and verbalizations: 924


# Neural Machine Translation (NMT) Model Definition 

## TODO: needs updating

## Idea:
1. Encoder: 
1.1 Input==Word Embedding (or Letter embedding); 
1.2 Output==Context Vector (that is: Encoding of sentence; contained in hidden state after having observed last embedding)

2. Decoder:
2.1 Input==Context Vector
2.2 Output==Probability distribution over output vocab (might be words or letters alternatively)

3. Seq2Seq model: Combining the two

## BERT Encoder

#### Tokenizer

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#### Model

In [12]:
bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)#.to('cuda')

## Decoder

### Soft Attention Model
This model implements the Soft Attention model presented in http://proceedings.mlr.press/v37/xuc15.pdf. 
1. Attention energies (i.e. energy per annotation vector) get computed: $e_{ti}=f_{att}(a_i,h_{t−1})$. Note that this formula implies that the Decoder's previous hidden state $h_{t-1}$ needs to be appended to each individual annotation vector $a_i$ before feeding their concatenation through a fully-connected layer $f_{att}$. 
2. Attention weights $\alpha$ get computed from the aforementioned energies: $\alpha_t = softmax(e_t)$, where $\alpha_{ti} = \frac{exp(e_{ti})}{\sum^L_{k=1} exp(e_{tk})}$.

Note: $t$ stands for time, while $i$ identifies the particular annotation vector currently under consideration.

In [13]:
class SoftAttention(nn.Module):
    
    def __init__(self, 
                 annotation_size,  # Tuple: (num_annotations, num_features_per_annotation)
                 hidden_len        # Number of nodes in Decoder's hidden state weight matrix
                ):
        
        super(SoftAttention, self).__init__()
        
        # Variables
        self.num_annotations = annotation_size[0]
        self.annotation_features = annotation_size[1]
        self.hidden_size = hidden_len
        
        # Layers
        self.attn = nn.Linear(self.annotation_features + self.hidden_size, 1, bias=True)
        self.softmax = nn.Softmax(dim=1)
        
        
    def forward(self, annotations, prev_hidden):
        
        # Repeat prev_hidded X times to append it to each of the annotation vectors (per batch element)
        repeated_hidden = torch.cat(
            [
                torch.repeat_interleave(hid, repeats=self.num_annotations, dim=0).unsqueeze(0)
                for hid in prev_hidden.split(1)
            ]
        )
        
        # Append previous hidden state to all annotation vectors (for each individual batch element)
        # Input to attention weight calculation
        input = torch.cat((annotations, repeated_hidden), dim=2)
        
        # Compute the relative attention scores per feaure (e_{ti}=f_{att}(a_i,h_{t−1}) from paper)
        energies = self.attn(input)
        
        # Compute final attention weights (i.e. alpha)
        attn_weights = self.softmax(energies)
        
        return attn_weights


### Decoder itself (employing Soft Attention)

In [14]:
class Decoder(nn.Module):
    
    def __init__(self, 
                 annotation_size,      # Size of annotation vectors produced by Encoder
                 out_vocab_size,       # How many words there are in the RDF-output language
                 embedding_dim,        # Length of a word embedding
                 hidden_dim,           # Nr hidden nodes
                 output_dim,           # Vocab size
                 bidirectional=False,  # Whether to have bidirectional GRU
                 n_layers=1,           # Nr layers in GRU
                 drop_prob=0.2         # Percent of node-dropouts
                ):
        
        super(Decoder, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.n_directions = 1 if not bidirectional else 2  # TODO: make use of it...
        
        self.attn = SoftAttention(annotation_size=annotation_size, hidden_len=hidden_dim)
        self.gru = nn.GRU(annotation_size[1]+embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
        
    def forward(self, 
                annotations,  # Static annotation vectors (for each batch element)
                embeddings,   # Word embeddings of most recently generated word (per batch element)
                h_old         # Previous hidden state per batch element
               ):
        
        attention_weights = self.attn(annotations, h_old).unsqueeze(2)
        weighted_annotations = annotations * annotation_weights
        context_vectors = torch.sum(weighted_annotations, dim=1)
        
        x = torch.cat((context_vectors, embeddings), dim=1)
        x = x.unsqueeze(1)
        
        out, h = self.gru(x, h_old)
        out = out.squeeze()
        out = self.softmax(self.fc(self.relu(out)))
        
        return out, h
    
    
    def init_hidden(self, annotation_vectors):
        # Mean of annotation vector per batch element
        # Assumes that number of hidden nodes == number annotation features
        hidden = torch.mean(permuted, dim=2).to(device)
        return hidden

## Train loop

In [15]:
# TODO: These are just some examples of fuctionalities to be used inside train loop
# Documentation: https://huggingface.co/transformers/model_doc/bert.html

# Sample mini-batch
mini_batch = ["Hello, my cat is cute", 
              "It is not", 
              "HERE I Am, Bye"]

# Tokenize sampled mini-batch sentences
inputs = tokenizer(mini_batch, 
                   return_tensors="pt",     # Return tensors in pt=PyTorch format
                   padding=True,            # Pad all sentences in mini-batch to have the same length
                   add_special_tokens=True) # Add "Start of sequence", "End of sequence", ... tokens. 

# Encode sentences: Pass tokenization output-dict-contents to model
outputs = bert_model(**inputs) 

# Retrieve hidden state to be passed into Decoder 
last_hidden_states = outputs.last_hidden_state  

# Print...
print(inputs)
print(last_hidden_states.shape)

{'input_ids': tensor([[  101,  7592,  1010,  2026,  4937,  2003, 10140,   102],
        [  101,  2009,  2003,  2025,   102,     0,     0,     0],
        [  101,  2182,  1045,  2572,  1010,  9061,   102,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0]])}
torch.Size([3, 8, 768])


In [30]:
def predict(x, 
            word_embeddings,        # Decoder's word embeddings
            word2idx,               # 
            idx2word,               # 
            encoder,                # 
            decoder,                # 
            tokenizer,              # 
            max_len=7,              # 
            batch_size=32,          # 
            targets=None,           # 
            return_textual=False    # Whether to return predictions in index-form (default) or as textual strings
           ):
    
    # Init documentation of predictions
    predicted_indices = torch.zeros([batch_size, max_len]) # Numeric
    if return_textual:
        predicted_words = ['']*batch_size
    
    # Tokenize sampled minibatch sentences
    inputs = tokenizer(minibatch, 
                       return_tensors="pt",     # Return tensors in pt=PyTorch format
                       padding=True,            # Pad all sentences in mini-batch to have the same length
                       add_special_tokens=True) # Add "Start of sequence", "End of sequence", ... tokens. 

    # Encode sentences: Pass tokenization output-dict-contents to model
    outputs = model(**inputs) 

    # Retrieve hidden state to be passed into Decoder as annotation vectors
    annotations = outputs.last_hidden_state  

    # Init Decoder's hidden state
    decoder.init_hidden(annotations)
    
    # Construct initial embeddings (start tokens)
    embeddings = word_embeddings(torch.zeros([batch_size]))
    
    for t in range(max_len):
        # Get decodings (aka prob distrib. over output vocab per batch element) for time step t
        prob_dist, hidden = decoder(annotations, # Static vector containing annotations per batch element 
                                    embeddings,  # Word embedding predicted last iteration (per batch element)
                                    h_old        # Decoder's hidden state of last iteratipn per batch element
                                    )

        # Get predicted word index from predicted probability distribution (per batch element)
        word_index = torch.max(prob_dist, dim=1)
        
        # Get corresponding word embedding (by index; per batch element)
        embedding = word_embeddings(word_index)
        
        # TODO: optional teacher forcing?

        # Record predicted words
        predicted_indices[:, t] = word_index
        
        # Record textual words if required
        if return_textual:
            
            # Get predicted word (per batch element)
            predicted_word = [idx2word(batch_element) for batch_element in word_index]
        
            for e in range(batch_size):
                predicted_words[e] += predicted_word[e]

        if targets:
            
            # Compute and document loss (averaged over all batch elements given current time step t)
            loss = nn.CrossEntropyLoss(prob_dist, targets[:, t])
            accumulated_loss += loss

            # Compute & back-propagate gradients
            loss.backward()
            
    ret_object = {
        'predicted_indices': predicted_indices,
    }
    
    if targets:
        ret_object['loss'] = accumulated_loss
    if return_textual:
        ret_object['predicted_words'] = predicted_words
        
    return ret_object 

In [28]:
def rdf_vocab(raw_vocab):
    vocab_count, word2idx, idx2word = 3, {'START': 0, 'PAD': 1, 'END': 2}, {0: 'START', 1: 'PAD', 2: 'END'}
    
    for partition in raw_vocab: # Different partitions with respect to nr or triples per sentence
        for train_instance in partition:
            triple = train_instance['triple']
            for part in triple:
                if token not in word2idx:
                    word2idx[token] = vocab_count
                    idx2word[vocan_count] = token
                    vocab_count += 1
    return vocab_count, word2idx, idx2word

In [27]:
def train(train_data, 
          val_data,  
          epochs, 
          minibatch_size=32,
          embedding_dim=300,
          eval_frequency=10 # Every how many epochs to run intermediate evaluation
         ):
    
    # Construct RDF vocab
    vocab_count, word2idx, idx2word = rdf_vocab(y_train)
    
    # Construct embeddings
    rdf_vocab = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
    
    # Define model
    encoder = bert_model
    decoder = Decoder(
        annotation_size=(8,768),    # Size of annotation vectors produced by Encoder
        out_vocab_size=vocab_count, # How many words there are in the RDF-output language
        embedding_dim=768,          # Length of a word embedding
        hidden_dim=256,             # Nr hidden nodes
        output_dim=256,             # Vocab size
    )
    
    # For both train and validation data & for all number of tuples per sentence 
    # (in [MIN_NUM_TRIPLES, MAX_NUM_TRIPLES]), get the nr of train-/test instances
    len_x_train = [len(train_set) for train_set in train]
    len_x_val = [len(val_set) for val_set in dev]
    
    # Development of both train- and validation losses over course of training
    train_losses, val_losses = [], []
    
    # Train
    for epoch in range(epochs):
        
        train_loss, eval_loss = 0., 0.
        
        # Reset gradients
        encoder.zero_grad()
        decoder.zero_grad()
        
        # Perform own train step for each nr of triples per sentence separately
        for i, nt in enumerate(range(MIN_NUM_TRIPLES, MAX_NUM_TRIPLES+1)):
            
            # Sample minibatch indices
            minibatch_idx = random.sample(population=range(len_x_train[i]), k=minibatch_size)
            
            # Number of tokens to be predicted (per batch element)
            num_preds = nt*3+1 # = nr triples * 3 + stop_token 
            
            # Construct proper minibatch
            inputs = [train_data[i][idx]['text'] for idx in minibatch]
            targets = torch.ones([minibatch_size, num_preds], dtype=int)
            
            # Predict
            ret_object = predict(inputs
                                rdf_vocab,        # Decoder's word embeddings
                                word2idx,               # 
                                idx2word,               # 
                                encoder,                # 
                                decoder,                # 
                                tokenizer,              # 
                                max_len=num_preds,      # Nr of tokens to be predicted
                                batch_size=32,          # 
                                targets=None,           # 
                                return_textual=False    # Whether to return predictions in index-form (default) or as textual strings
                               ):
            
            train_loss += ret_object['loss']
            
        # Apply gradients
        
        
        # Intermediate evaluation
        
        # Save losses
        
    return loss_train, loss_eval, encoder, decoder

## TODO: Make only a train function and a predict function, but skip Seq2seq ???

# Train

In [ ]:
train_loss, eval_loss, encoder, decoder = train(x_train, y_train, x_val, y_val, epochs) # TODO

# Test